## Dimensionality Reduction: Based on model performance

In [93]:
import os
import pandas  as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib   import Path
from sklearn.preprocessing     import StandardScaler
from sklearn.model_selection   import train_test_split
from sklearn.metrics           import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.ensemble          import RandomForestClassifier
from sklearn.ensemble          import GradientBoostingRegressor
from sklearn.ensemble          import RandomForestRegressor
from sklearn.linear_model      import Lasso
from sklearn.linear_model      import LassoCV
from sklearn.linear_model      import LogisticRegression
from sklearn.linear_model      import LinearRegression

In [2]:
# set root directory
path_root = Path("C:/Users/giann/data-science-core")
os.chdir(path_root)
print(f'- Root directory = {os.getcwd()}')

- Root directory = C:\Users\giann\data-science-core


In [12]:
# import dataset
path_dataset = path_root / 'dataset/PimaIndians.csv'
data  = pd.read_csv(path_dataset)
data.head()

,pregnant,glucose,diastolic,triceps,insulin,bmi,family,age,test
0,1,89,66,23,94,28.1,0.167,21,negative
1,0,137,40,35,168,43.1,2.288,33,positive
2,3,78,50,32,88,31.0,0.248,26,positive
3,2,197,70,45,543,30.5,0.158,53,positive
4,1,189,60,23,846,30.1,0.398,59,positive


In [13]:
X = data.drop('test', axis = 1)
y = data['test']

In [23]:
# scale data
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.3, stratify = y, random_state = 42)

C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


### Drop features using Recursive Feature Elimination (RFE)
- Dropping one feature at a time is **safest** since it will affect other feature's coef
- RFE can be **wrapped** to any model that produces feature coefficient
- RFE drop first the model and then drop the feature with the weakest coefficients

In [21]:
# Create the RFE with a LogisticRegression estimator and 3 features to select
rfe = RFE(estimator = LogisticRegression(solver = 'lbfgs'), n_features_to_select=3, verbose=1)
# Fit the eliminator to the data
rfe.fit(X_train, y_train)

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.


RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
  n_features_to_select=3, step=1, verbose=1)

In [22]:
# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe.ranking_)))

{'pregnant': 3, 'glucose': 1, 'diastolic': 6, 'triceps': 4, 'insulin': 5, 'bmi': 1, 'family': 2, 'age': 1}


Values of 1 on `rfe.rarking` means that the feature was kept till the end, and viceversa. 

In [24]:
# Print the features that are not eliminated
print(X.columns[rfe.support_])

Index(['glucose', 'bmi', 'age'], dtype='object')


In [26]:
# Calculates the test set accuracy
acc = accuracy_score(y_test, rfe.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

80.5% accuracy on test set.


### Tree-based feature selection
- Radom forest does not requirte feature scaling.
- it is important also in this case tuo use `RFE` since dropping one feature might give more weight to another one.

In [8]:
# import dataset
path_dataset = path_root / 'dataset/PimaIndians.csv'
data  = pd.read_csv(path_dataset)
data.head()

,pregnant,glucose,diastolic,triceps,insulin,bmi,family,age,test
0,1,89,66,23,94,28.1,0.167,21,negative
1,0,137,40,35,168,43.1,2.288,33,positive
2,3,78,50,32,88,31.0,0.248,26,positive
3,2,197,70,45,543,30.5,0.158,53,positive
4,1,189,60,23,846,30.1,0.398,59,positive


In [9]:
X = data.drop('test', axis = 1)
y = data['test']
# Perform a 75% training and 25% test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [10]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state = 0, n_estimators = 10)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [11]:
# Print the importances per feature
print(dict(zip(X.columns, rf.feature_importances_.round(2))))

{'pregnant': 0.09, 'glucose': 0.21, 'diastolic': 0.08, 'triceps': 0.11, 'insulin': 0.13, 'bmi': 0.09, 'family': 0.12, 'age': 0.16}


In [12]:
# Calculate the test set accuracy
acc = accuracy_score(y_test, rf.predict(X_test))
# Print accuracy
print("{0:.1%} accuracy on test set.".format(acc)) 

77.6% accuracy on test set.


The random forest model gets 78% accuracy on the test set and `glucose` is the most important feature.

In [14]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15
# Apply the mask to the feature dataset X
reduced_X = X.loc[:, mask]
# prints out the selected column names
print(reduced_X.columns)

Index(['glucose', 'age'], dtype='object')


Note that in this way we might **discard** important feature. The **right way** is using the `RFE` as before.

In [19]:
# Wrap the feature eliminator around the random forest model
rfe = RFE(estimator = RandomForestClassifier(random_state = 0, n_estimators = 10), n_features_to_select = 2, verbose = 1, step = 2)
# Fit the model to the training data
rfe.fit(X_train, y_train)

Fitting estimator with 8 features.
Fitting estimator with 6 features.
Fitting estimator with 4 features.


RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
  n_features_to_select=2, step=2, verbose=1)

In [20]:
# Create a mask
mask = rfe.support_
# Apply the mask to the feature dataset X and print the result
reduced_X = X.loc[:, mask]
print(reduced_X.columns)

Index(['glucose', 'insulin'], dtype='object')


Note that in this case the feature have been **changed**!!! 

### Dimensionality Reduction for regressor problems via Lasso

In [31]:
# import dataset
path_dataset = path_root / 'dataset/ANSUR_II_FEMALE.csv'
data  = pd.read_csv(path_dataset)
# exclude categorical variable
data = data.select_dtypes(exclude = 'object')

In [32]:
# split data
X = data.drop('BMI', axis = 1)
y = data['BMI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [33]:
# Set the test size to 30% to get a 70-30% train test split
scaler = StandardScaler()
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)

C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [41]:
# Create the Lasso model
la = Lasso(max_iter = 3000)
# Fit it to the standardized training data
la.fit(X_train_std, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=3000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [42]:
# Evaluate model performance
# Scale the test set
X_test_std = scaler.transform(X_test)
# Calculate the coefficient of determination (R squared) on the scaled test set
r_squared = la.score(X_test_std, y_test)
print("o The model can predict {0:.1%} of the variance in the test set.".format(r_squared))

o The model can predict 80.5% of the variance in the test set.


C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [43]:
# Create a list that has True values when coefficients equal 0
zero_coef = la.coef_ == 0
# Take the sum of this list
n_ignored = sum(zero_coef)
print("o The model has ignored {} out of {} features.".format(n_ignored, len(la.coef_)))

o The model has ignored 86 out of 93 features.


Remember to tune $alpha$ to improve the performance. For this reason it is best to use the `LassoCV` class to automatically tune the best alpha.

**LassoCV**

In [47]:
# import dataset
path_dataset = path_root / 'dataset/ANSUR_II_FEMALE.csv'
data  = pd.read_csv(path_dataset)
# exclude categorical variable
data = data.select_dtypes(exclude = 'object')

In [48]:
# split data
X = data.drop('BMI', axis = 1)
y = data['BMI']

In [50]:
# Scale and split
scaler = StandardScaler()
X_std  = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.3, random_state = 42)

C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [54]:
# Create and fit the LassoCV model on the training set
lcv = LassoCV(cv = 5)
lcv.fit(X_train, y_train)
print('o Optimal alpha = {0:.3f}'.format(lcv.alpha_))

o Optimal alpha = 0.003


In [55]:
# Calculate R squared on the test set
r_squared = lcv.score(X_test, y_test)
print('o The model explains {0:.1%} of the test set variance'.format(r_squared))

o The model explains 99.5% of the test set variance


In [56]:
# Create a mask for coefficients not equal to zero
lcv_mask = lcv.coef_ != 0
print('o {} features out of {} selected'.format(sum(lcv_mask), len(lcv_mask)))

o 54 features out of 93 selected


## Ensemble models for extra votes
One can combine multiple models and carry out feature selection using a **metamodel**. Here we use **Lasso Regression**, **Random Forest** and **Gradient Boosting** to perform feature selection. 

In [109]:
# import dataset
path_dataset = path_root / 'dataset/ANSUR_II_FEMALE.csv'
data  = pd.read_csv(path_dataset)
# exclude categorical variable
data = data.select_dtypes(exclude = 'object')

In [110]:
# split data
X = data.drop('BMI', axis = 1)
y = data['BMI']

In [111]:
# Scale and split
scaler = StandardScaler()
X_std  = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


**STEP 1**: Let us train a model one by one;

**Lasso Regression**

In [112]:
lcv = LassoCV(cv = 5)
lcv.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [113]:
r_squared = lcv.score(X_test, y_test)
print('o The model explains {0:.1%} of the test set variance'.format(r_squared))

o The model explains 99.0% of the test set variance


In [114]:
# Create a mask for coefficients not equal to zero
lcv_mask = lcv.coef_ != 0
print('o {} features out of {} selected'.format(sum(lcv_mask), len(lcv_mask)))
n_feature_selected = sum(lcv_mask)

o 40 features out of 93 selected


**Gradient Boosting**: use `RFE` and select the same number of features obtained with `LassoCV`. 

In [115]:
# Select 10 features with RFE on a GradientBoostingRegressor, drop 3 features on each step
rfe_gb = RFE(estimator = GradientBoostingRegressor(n_estimators = 10), n_features_to_select = n_feature_selected, step = 5, verbose = 1)
rfe_gb.fit(X_train, y_train)

Fitting estimator with 93 features.
Fitting estimator with 88 features.
Fitting estimator with 83 features.
Fitting estimator with 78 features.
Fitting estimator with 73 features.
Fitting estimator with 68 features.
Fitting estimator with 63 features.
Fitting estimator with 58 features.
Fitting estimator with 53 features.
Fitting estimator with 48 features.
Fitting estimator with 43 features.


RFE(estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
  n_features_to_select=40, step=5, verbose=1)

In [116]:
# Calculate the R squared on the test set
r_squared = rfe_gb.score(X_test, y_test)
print('The model can explain {0:.1%} of the variance in the test set'.format(r_squared))

The model can explain 75.2% of the variance in the test set


In [117]:
# Assign the support array to gb_mask
gb_mask = rfe_gb.support_
print('o {} features out of {} selected'.format(sum(gb_mask), len(gb_mask)))

o 40 features out of 93 selected


**Random Forest**: use `RFE` and select the same number of features obtained with `LassoCV`.

In [118]:
# Select 10 features with RFE on a RandomForestRegressor, drop 3 features on each step
rfe_rf = RFE(estimator = RandomForestRegressor(n_estimators = 10), 
             n_features_to_select = n_feature_selected, step = 5, verbose = 1)
rfe_rf.fit(X_train, y_train)

Fitting estimator with 93 features.
Fitting estimator with 88 features.
Fitting estimator with 83 features.
Fitting estimator with 78 features.
Fitting estimator with 73 features.
Fitting estimator with 68 features.
Fitting estimator with 63 features.
Fitting estimator with 58 features.
Fitting estimator with 53 features.
Fitting estimator with 48 features.
Fitting estimator with 43 features.


RFE(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
  n_features_to_select=40, step=5, verbose=1)

In [119]:
# Calculate the R squared on the test set
r_squared = rfe_rf.score(X_test, y_test)
print('The model can explain {0:.1%} of the variance in the test set'.format(r_squared))

The model can explain 95.7% of the variance in the test set


In [120]:
# Assign the support array to gb_mask
rf_mask = rfe_rf.support_
print('o {} features out of {} selected'.format(sum(rf_mask), len(rf_mask)))

o 40 features out of 93 selected


**STEP 2**: **Mean voting** assess the votes of each model. The feature selection depend on how conservative one want to be. In this case we select the features that have been selected by all the three models. 

In [121]:
# Sum the votes of the three models
votes = np.sum([lcv_mask, rf_mask, gb_mask], axis=0)
print(votes)

[2 2 1 3 3 1 1 1 3 1 3 1 2 1 1 3 3 2 0 1 3 2 1 3 2 2 2 1 1 0 0 0 1 1 0 0 0
 2 2 0 1 0 0 0 0 1 0 0 0 0 1 2 1 2 0 1 1 0 0 3 0 2 0 0 0 3 0 1 1 1 1 1 1 2
 3 3 3 3 0 1 1 2 1 2 1 3 3 1 2 0 2 3 2]


In [122]:
# Create a mask for features selected by all 3 models
meta_mask = votes >= 3
meta_mask

array([False, False, False,  True,  True, False, False, False,  True,
       False,  True, False, False, False, False,  True,  True, False,
       False, False,  True, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False,  True,  True,  True,  True, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False,  True, False])

**STEP 3**: Apply the dimensionality reduction on `X` and fit a model

In [128]:
# split data
X = data.drop('BMI', axis = 1)
y = data['BMI']

In [129]:
X_reduced = X.loc[:, meta_mask]

In [130]:
# Plug the reduced dataset into a linear regression pipeline
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=0)

In [131]:
lm = LinearRegression()
lm.fit(scaler.fit_transform(X_train), y_train)

C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [132]:
r_squared = lm.score(scaler.transform(X_test), y_test)
print('The model can explain {0:.1%} of the variance in the test set using {1:} features.'.format(r_squared, len(lm.coef_)))

The model can explain 98.6% of the variance in the test set using 17 features.


C:\Users\giann\Anaconda3\envs\data-science\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
